# Projet 1 (semaines 1 à 4) : Reversi 

## Semaine 2 : Probabilités Conditionnelles et importance des coins


Un des points important pour construire un bot capable de jouer à un jeu est de d'être capable d'évaluer une position, i.e. donner une probabilité de gagner selon l'état du jeu. Cette quantité est une probabilité conditionnelle à l'état courant du jeu. Dans un jeu combinatoire comme le reversi, il est impossible de calculer cette probabilité pour tous les états du jeu, même en éliminant les symétries (cf partie 1, trop de configurations possibles), mais il est possible de l'estimer. Par ailleurs, certaines positions sont plus intéressantes que d'autres. 

Le but de cette partie est d'explorer l'importance de la prise des coins du plateaux en calculant un certain nombre de probabilités conditionnelles de gain.  

In [1]:
%load_ext autoreload
%autoreload 2
from reversi import Reversi, play_game
import tme2
import numpy as np


## II.0 Préambule 

Les coins font partis des cases les plus importantes du plateau, car une fois capturés ils ne peuvent plus être pris par l'adversaire. 

On veut estimer la probabilité de gagner selon que l'on a 0 coins, 1, 2, 3 ou 4 coins pris en fin de jeu (et l'adversaire les autres coins). Pour cela, vous allez engendrer un grand nombre de parties jouées au hasard jusqu'au bout, puis compter le nombre de configurations qui correspondent à chaque situation. 

Comment en déduire alors  $P(gagner|~n~coins~pris)$ ? Quel lien/différence avec $P(gagner, ~n~coins~pris)$ ?


Vous pouvez stocker les parties sous la forme de tableaux numpy mais cela gaspillerait beaucoup de mémoire. Le mieux est de les stocker comme en première semaine sous la forme d'un couple d'entiers grâce à la fonction *board_to_int()*. Dans ce cas, plutôt que de décompresser la représentation pour analyser le plateau, il est plus facile de travailler directement sur l'encodage binaire du plateau. 

Rappel : le plateau est encodé par deux entiers *(p1,p2)* avec chacun autant de bits que de cases (64 bits dans notre cas); l'entier *p1* encode les positions des pions du joueur 1, l'entier *p2* celui du deuxième joueur. Les bits correspondants aux cases occupées sont mis à 1, les autres à 0. Ainsi l'entier *1* représente un plateau avec seule la case *(0,0)* d'occupé, l'entier *3* avec les deux premières cases d'occupés, etc. Pour savoir si une case est occupée, il suffit de faire un **AND** logique entre l'entier représentant la case et le plateau (ce qu'on appelle un masque). De même, pour savoir le nombre de cases d'une configuration donnée incluse dans un état du plateau, il suffit de faire un **AND** logique entre l'entier représentant la configuration et celui de l'état du plateu, et de compter le nombre de bits de l'entier résultant (avec la fonction *bit_count()* des entiers de python par exemple).

### II.1 Génération des masques

Codez une fonction **get_mask_one(x,y)** qui permet de renvoyer le masque (l'entier) correspondant à un plateau occupé uniquement par un pion en *(x,y)* (regardez pour cela comment est codé  la fonction **board_to_int()** de la classe **Reversi**, ou plus simple mais plus long, créez un jeu vide, mettez la case voulue à 1 et utilisez **board_to_int**).  Testez votre fonction avec le code ci-dessous.

In [2]:
# Création d'un jeu
game = Reversi(size=8)
# Initialisation à 0
game.board = np.zeros((8,8))
# Initialisation d'une case
game.board[3,4] = 1
# Bitboard des deux joueurs
p1,p2 = game.board_to_int()
# Le masque ne contient qu'une case, donc le résultat du AND doit avoir qu'un bit à 1
assert((p1 & tme2.get_mask_one(3,4)).bit_count() == 1)
# Autre manière de tester, le résultat du AND doit être le même que le masque
assert((p1 & tme2.get_mask_one(3,4)) == tme2.get_mask_one(3,4) )
# On s'assure que pour une autre case le résultat doit être 0
assert(p1 & tme2.get_mask_one(3,5) == 0)



### II.2 Plusieurs cases

Codez une fonction **get_mask(l)** avec **l** une liste de cases **(x,y)** qui permet de renvoyer le masque correspondant (il suffit d'additionner les masques obtenus avec la fonction précédente).

In [3]:
# Réinitialisation à 0
game.board = np.zeros((8,8))
# Initialisation de deux cases
game.board[3,5] = 1
game.board[2,2] = 1
# Bitboard des deux joueurs
p1,p2 = game.board_to_int()
# Le masque contient deux cases, donc le résultat du AND doit avoir deux bits à 1
assert((p1 & tme2.get_mask([(3,5),(2,2)])).bit_count()==2)
# Autre manière de tester, le résultat du AND doit être le même que le masque
assert((p1 & tme2.get_mask([(3,5),(2,2)])) == tme2.get_mask([(3,5),(2,2)]))
# Si on a qu'une seule case, alors le résultat du AND doit être 1
assert((p1 & tme2.get_mask([(3,5),(2,5)])).bit_count()==1)



### I.3 Rollout et simulation

Nous aurons besoin de simuler une partie aléatoire à partir d'une configuration donnée. Pour cela, codez une fonction **rollout(game,turn=1,nb_moves=100)** avec **game** un jeu quelconque (pas forcément dans une configuration initiale), **turn** le joueur à qui c'est le tour de jouer (1 ou -1), et **nb_moves** 
le nombre de coups à effectuer. Cette fonction fait jouer au hasard les 2 joueurs jusqu'à ce que **nb_moves** soient joués (un nombre de coups supérieur à 64 assure que la partie est finie) et renvoie le jeu dans la configuration finale.

Codez une fonction **simu_mc(game,turn=1,nb_simu=10000,nb_moves=100)** qui à partir d'un même état **game**
engendre **nb_simu** jeux aléatoires en faisant appel à **rollout**  et retourne la liste des configurations finales sous la forme renvoyée par **board_to_int()**.


In [4]:
game2 = Reversi(size=8)
tme2.rollout(game2).print_board()
tme2.simu_mc(game2)

  0 1 2 3 4 5 6 7
0 O O O O O O O O
1 O O O O O X O O
2 O O O X X O X O
3 O O X O X O O O
4 O X O O O O O O
5 O O X O X X O O
6 O O O O O O X X
7 O O O O O O O O


[(18392643694645207039, 54100379064344576),
 (18392643694645207039, 54100379064344576),
 (18392643694645207039, 54100379064344576),
 (18392643694645207039, 54100379064344576),
 (18392643694645207039, 54100379064344576),
 (18392643694645207039, 54100379064344576),
 (18392643694645207039, 54100379064344576),
 (18392643694645207039, 54100379064344576),
 (18392643694645207039, 54100379064344576),
 (18392643694645207039, 54100379064344576),
 (18392643694645207039, 54100379064344576),
 (18392643694645207039, 54100379064344576),
 (18392643694645207039, 54100379064344576),
 (18392643694645207039, 54100379064344576),
 (18392643694645207039, 54100379064344576),
 (18392643694645207039, 54100379064344576),
 (18392643694645207039, 54100379064344576),
 (18392643694645207039, 54100379064344576),
 (18392643694645207039, 54100379064344576),
 (18392643694645207039, 54100379064344576),
 (18392643694645207039, 54100379064344576),
 (18392643694645207039, 54100379064344576),
 (18392643694645207039, 54100379

### II.4 Estimation de la probabilité de gagner avec un certain nombre de coins


Codez une fonction **estime_coin(simus,nb_coins)** qui renvoie la probabilité de gagner sachant que l'on occupe exactement **nb_coins** à la fin de la partie. Pour cela, vous pouvez utiliser les fonctions de générations de masque en créant un masque pour les 4 coins, puis compter le nombre de partie dans simus qui vérifient les conditions aux coins. 

Quelle différence avec la probabilité de gagner et que l'on occupe exactement **nb_coins** ?

In [5]:
game = Reversi()
simus = tme2.simu_mc(game)

In [6]:
tme2.estime_coins(simus,1)

0.255

In [7]:
tme2.estime_coins(simus,2)

0.369

In [8]:
tme2.estime_coins(simus,3)

0.254

### I.4 Estimation de la probabilité de gagner en prenant un coin

Le résultat précédent ne nous renseigne pas forcément sur l'importance d'occuper un coin en premier. 

Codez une fonction **cdt_one_corner(game)** qui permet de tester si un coin est occupé par le premier joueur et aucun coin par le deuxième joueur.

Codez une fonction **play_until(game,cdt,turn)** qui déroule une partie au hasard à partir de l'état **game** sachant que c'est au joueur **turn** de jouer jusqu'à ce que la condition **cdt** soit vrai : **cdt** est une fonction avec la même signature que  **cdt_one_corner**. Cette fonction nous permettra donc de jouer une partie jusqu'à ce qu'une condition soit remplie (ou que le jeu se finisse sans que la condition ne soit remplie). 

Codez enfin une fonction **find_game(cdt)** qui initialise une partie et la joue jusqu'à ce que la condition **cdt** soit remplie; si la partie se finit avant que la condition ne soit remplie, alors une nouvelle partie est lancée jusqu'à en trouver une satisfiant la condition.


Codez enfin une fonction **compute_win_cdt(cdt,nb_games,nb_simu)** qui permet d'estimer par Monte-Carlo la probabilité de gain sachant que la condition **cdt** est rencontrée. Il n'est pas souhaitable de le faire que sur une configuration donnée, car l'estimation serait alors biaisée par cette configuration. Il faudra donc tirer **nb_games** configurations qui correspondent à **cdt** (à l'aide de **find_game**, c'est la boucle externe de la fonction), et pour chaque configuration, utiliser **simu_mc** pour exécuter **nb_simu** à partir de cette configuration. Le résultat de l'estimation est la moyenne de tous les résultats.

Ce nombre vous semble-t-il stable ?

In [ ]:
game = tme2.find_game(tme2.cdt_one_corner)
game.print_board()

In [ ]:
tme2.compute_win_cdt(tme2.cdt_one_corner,100,100)